In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor,ToPILImage,Normalize
from utils.dataset import LoadFilesDataset
from utils.model import construct_style_loss_model,construct_decoder_from_encoder,AdaIN
from utils.losses import content_gatyes,style_gatyes,style_mmd_polynomial,adaIN
from utils.utility import video_to_frame_generator,video_to_frames,normalize
import cv2
import os
from copy import deepcopy
import torch.utils.data as data
from torchvision.models import vgg19,VGG19_Weights
from PIL import Image
import torch.optim as optim
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On device: ", device)

On device:  cuda


In [14]:
# keep 16:9 ratio
IMAGE_SIZE = (512,288)
CONTENT_VIDEO_PATH = "./content_video/content_video.mp4"
STYLE_VIDEO_PATH = "./style_video/style_video.mp4"

In [ ]:
# load a generator that returns each video frame
content_frame_generator = video_to_frame_generator(CONTENT_VIDEO_PATH,IMAGE_SIZE)
style_frame_generator = video_to_frame_generator(STYLE_VIDEO_PATH,IMAGE_SIZE)

In [15]:
CONTENT_FRAME_SAVE_PATH = "./content_frames/"
STYLE_FRAME_SAVE_PATH = "./style_frames/"

In [ ]:
# save the single frames in a path to load later
video_to_frames(content_frame_generator, CONTENT_FRAME_SAVE_PATH)
video_to_frames(style_frame_generator, STYLE_FRAME_SAVE_PATH)

In [16]:
CONTENT_LAYERS = [6]
STYLE_LAYERS = [9,12,14]
CONTENT_LAYERS_WEIGHTS = [1.0]
STYLE_LAYERS_WEIGHTS = [1.0,1.0,1.0]

In [17]:
# load standard vgg19 model
vgg = vgg19(VGG19_Weights.DEFAULT)

# remove classification head
vgg = vgg.features

# prepend a normalization layer
vgg = nn.Sequential(Normalize(mean = (0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)), *vgg)

style_loss_model = construct_style_loss_model(vgg,CONTENT_LAYERS,STYLE_LAYERS)
style_loss_model = style_loss_model.eval()
style_loss_model.requires_grad_(False)
style_loss_model = style_loss_model.to(device)

# as an encoder for our feedforward model we will just use the already existing vgg layers up to some point
vgg_encoder = vgg[:8]

# based on that we build a decoder that reverses our encoder and matches the shapes through interpolation
decoder = construct_decoder_from_encoder(vgg_encoder.cpu(),3,*IMAGE_SIZE)

range(0, 15)


In [18]:
BATCH_SIZE = 12

In [19]:
content_dataloader =data.DataLoader(LoadFilesDataset(CONTENT_FRAME_SAVE_PATH,batch_size= BATCH_SIZE),batch_size=BATCH_SIZE)
style_dataloader = data.DataLoader(LoadFilesDataset(STYLE_FRAME_SAVE_PATH,batch_size= BATCH_SIZE),batch_size=BATCH_SIZE)

In [20]:
class StyleModel(nn.Module):
    def __init__(self, encoder, decoder) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.adain = AdaIN()

    def forward(self, image, image_style):
        f = self.encoder(image)
        f_style = self.encoder(image_style)

        f = self.adain(f,f_style)

        image = self.decoder(f)

        return image

In [21]:
style_model = StyleModel(vgg_encoder,decoder).to(device)
optimimizer = optim.Adam(decoder.parameters(),lr=0.001)

In [22]:
EPOCHS = 10000
ACCUM_GRAD = 1
PRINT_STATS_EVERY = 100
STYLE_WEIGHT = 1.0
LOSS_CONTENT = content_gatyes
LOSS_STYLE = adaIN
SAVE_PATH = "./adain_save/"

In [23]:
os.makedirs(SAVE_PATH,exist_ok = True)

content_loss_aggregator = []
style_loss_aggregator = []

for epoch in tqdm(range(EPOCHS)):
    optimimizer.zero_grad()

    for _ in range(ACCUM_GRAD):
        content_img = next(iter(content_dataloader)).to(device)
        style_img = next(iter(style_dataloader)).to(device)

        with torch.no_grad():
            _,content_features_target,_ = style_loss_model((content_img,[],[]))
            _,_,style_features_target = style_loss_model((style_img,[],[]))
        

        prediction = style_model(content_img,style_img)

        _,content_features,style_features= style_loss_model((prediction,[],[]))

        content_loss = 0.0
        for f,f_target,weight in zip(content_features,content_features_target, CONTENT_LAYERS_WEIGHTS):
            content_loss += weight*LOSS_CONTENT(normalize_cw(f,f_target)).mean()

        style_loss = 0.0
        for f,f_target,weight in zip(style_features,style_features_target, STYLE_LAYERS_WEIGHTS):
            style_loss += weight*LOSS_STYLE(normalize_cw(f,f_target)).mean()

        style_loss *= STYLE_WEIGHT

        content_loss_aggregator.append(content_loss.detach().cpu())
        style_loss_aggregator.append(style_loss.detach().cpu())

        loss = content_loss+style_loss

        loss.backward()

    optimimizer.step()  

    if epoch%PRINT_STATS_EVERY == 0 and not epoch == 0:
        print("Ending epoch: ", str(epoch), " with content loss: ", torch.stack(content_loss_aggregator).mean().numpy().item(),  " and style loss: ", torch.stack(style_loss_aggregator).mean().numpy().item())
        img = content_img[0]
        style_img = style_img[0]

        with torch.no_grad():   
            img = style_model(img.unsqueeze(0), style_img.unsqueeze(0))
        img = ToPILImage()(img.cpu().squeeze(0).permute(0,2,1))
        img.save(SAVE_PATH + "example_output_model.jpg")
        torch.save(style_model.state_dict(), SAVE_PATH + "model_weights.pth")
        torch.save(optimimizer.state_dict(), SAVE_PATH + "optim_weights.pth")


    
    

  1%|          | 100/10000 [00:56<1:25:28,  1.93it/s]

Ending epoch:  100  with content loss:  1.2336143255233765  and style loss:  0.5468908548355103


  2%|▏         | 201/10000 [01:46<1:29:39,  1.82it/s]

Ending epoch:  200  with content loss:  1.0675320625305176  and style loss:  0.4527081251144409


  3%|▎         | 301/10000 [02:38<1:31:29,  1.77it/s]

...

Ending epoch:  5900  with content loss:  0.7058200240135193  and style loss:  0.25149938464164734


 60%|██████    | 6001/10000 [48:23<39:04,  1.71it/s]

Ending epoch:  6000  with content loss:  0.7051131129264832  and style loss:  0.25117090344429016


 61%|██████    | 6088/10000 [49:07<31:34,  2.07it/s]


KeyboardInterrupt: 